In [1]:
from transformers import (
    AutoTokenizer,
    BitsAndBytesConfig,
)
from peft import AutoPeftModelForSequenceClassification
from datasets import load_dataset
import torch
from tqdm.notebook import tqdm
import pandas as pd
import os

/fs01/home/dadsetan/condaenvs/lora-finetuning-mia/lib/python3.10/site-packages/transformers/utils/generic.py:441: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(
/fs01/home/dadsetan/condaenvs/lora-finetuning-mia/lib/python3.10/site-packages/transformers/utils/generic.py:309: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(


In [2]:
tokenizer = AutoTokenizer.from_pretrained("/model-weights/Llama-2-7b-hf", return_tensors="pt", 
                                          model_max_length=150)

tokenizer.pad_token = tokenizer.eos_token

In [3]:
orig_dataset = load_dataset("parquet", data_files="data_balanced.parquet")

dataset = orig_dataset.rename_column("Lable", "label").rename_column("Text", "text")
dataset = dataset["train"].train_test_split(test_size=0.1, seed=42)

def change_labels(example):
  label = example["label"]
  n_label = 0 if label == "negative" else 1 if label == "neutral" else 2
  example["label"] = n_label
  return example

dataset = dataset.map(change_labels)


Map:   0%|          | 0/8183 [00:00<?, ? examples/s]

Map:   0%|          | 0/910 [00:00<?, ? examples/s]

In [11]:
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)
id2label = {0: "negative", 1: "neutral", 2: "positive"}
label2id = {"negative": 0, "neutral": 1, "positive": 2}
device = "cuda"

project_root = "/projects/privacyllm"

models = {
    "tweet_shield_anonymized_trained_model": [
        "checkpoint-1364",
        "checkpoint-2728",
        "checkpoint-4092",
        "checkpoint-5456",
        "checkpoint-6820",
        "checkpoint-8184",
        "checkpoint-9548",
        "checkpoint-10912",
        "checkpoint-12276",  
    ],
    "tweet_shield_dp_trained_model": [
        "checkpoint-1363",
        "checkpoint-2726",
        "checkpoint-4089",
        "checkpoint-5452",
        "checkpoint-6815",
        "checkpoint-8178",
        "checkpoint-9541" ,
        "checkpoint-10904",
        "checkpoint-12267",
    ],
    "tweet_shield_normal_trained_model": [
        "checkpoint-1364",
        "checkpoint-2728",
        "checkpoint-4092",
        "checkpoint-5456",
        "checkpoint-6820",
        "checkpoint-8184",
        "checkpoint-9548",
        "checkpoint-10912",
        "checkpoint-12276",
        "checkpoint-13640"
    ]
}

In [5]:
def log_acc(algorithm, checkpoint, epoch_no, acc):
    df = pd.DataFrame({
        "algorithm": [algorithm],
        "checkpoint": [checkpoint],
        "epoch_no": [epoch_no],
        "acc": [acc],
    })
    path = "results.csv"
    df.to_csv(path, mode="a", header=not os.path.exists(path), index=False)

In [26]:
def get_acc(model):
    correct = 0
    for data in tqdm(dataset["test"]):
        input_ids = tokenizer(data["text"], return_tensors='pt')["input_ids"].to("cuda")
    
        logits = model(input_ids=input_ids)["logits"]
        label = torch.argmax(logits,dim=-1)[0].item()

        correct += (label == data["label"])
    return float(correct)/len(dataset["test"])

In [28]:
for algorithm in models:
    for i, checkpoint in enumerate(models[algorithm]):
        print(f"Computing for algorithm {algorithm} and echeckpoitn {checkpoint} and epoch {i}") 
        path = f"{project_root}/{algorithm}/{checkpoint}"
        
        model = AutoPeftModelForSequenceClassification.from_pretrained(
            path,
            num_labels=3,
            quantization_config=bnb_config
        )
        model = model.to(device)
        model.eval()
        model.config.pad_token_id = model.config.eos_token_id
        
        accuracy = get_acc(model)
        log_acc(algorithm, checkpoint, i, accuracy)        

Computing for algorithm tweet_shield_anonymized_trained_model and echeckpoitn checkpoint-1364 and epoch 0


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Some weights of LlamaForSequenceClassification were not initialized from the model checkpoint at /model-weights/Llama-2-7b-hf and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


  0%|          | 0/910 [00:00<?, ?it/s]

819
Computing for algorithm tweet_shield_anonymized_trained_model and echeckpoitn checkpoint-2728 and epoch 1


...

839
Computing for algorithm tweet_shield_normal_trained_model and echeckpoitn checkpoint-13640 and epoch 9


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Some weights of LlamaForSequenceClassification were not initialized from the model checkpoint at /model-weights/Llama-2-7b-hf and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


  0%|          | 0/910 [00:00<?, ?it/s]

839
